In [1]:
import csv 
from datetime import timedelta, datetime
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt 
from matplotlib.dates import MonthLocator 
from matplotlib.dates import DateFormatter

In [76]:
from main import Entries

e = Entries("private/ViewingActivity.csv")
df = e.df 
df['startdate'] = [d.date() for d in df['start']]
df['starttime'] = [d.hour for d in df['start']]
df['year'] = [d.year for d in df['start']]
df['month'] = [(d.year, d.month) for d in df['start']]
df['weekday'] = [d.weekday() for d in df['start']]
USERS = df['user'].unique()

In [79]:


def plot_thing(yaxis='startdate', datetime=datetime(2020,1,1), size=(16,5)):
    df2 = df[df['start'] > datetime] 
    df2 = df2[['user','duration_in_seconds',yaxis]]
    df2['sum'] = df2.groupby(['user', yaxis])['duration_in_seconds'].transform('sum')
    df2 = df2.drop_duplicates(subset=['user', yaxis])
    pivot_df = df2.pivot(index=yaxis, columns='user', values='sum')
    fig, ax = plt.subplots( figsize=size)

    pivot_df.loc[:,USERS].plot(kind='bar', stacked=True, ax=ax)
    # Sadly pandas hates these formatters
    #ax.xaxis.set_major_locator(MonthLocator(interval=3))
    #ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
    ax.set_ylabel("Time watched in seconds")
    ax.set_xlabel(yaxis)
    #print(hi.get_xticks())

    #hi.yaxis.
    plt.xticks(rotation=70)
    plt.tight_layout()
    plt.show()
    
#plot_thing('month', datetime(2015,1,1), size=(10,5))

In [81]:
def get_show(s):
    test = s.split(':')
    return test[0]
def print_watch_hours(selected_user=None,topk=10):
    df2 = df[df['start'] > datetime(2014,1,1)] 
    df2['real_title'] = [get_show(s) for s in df2['title']]
    df2 = df2[['real_title','user','start','duration_in_seconds']]
    
    if selected_user is not None: df2 = df2[df2['user']==selected_user]



    df2['watched_show'] = df2.groupby([ 'real_title'])['duration_in_seconds'].transform('sum')
    #df2.drop_duplicates(subset=[ 'real_title'])
    df2['watched_show_hours'] = df2['watched_show'] / 60 / 60
    yes = df2[['real_title','watched_show_hours']].drop_duplicates().sort_values(by='watched_show_hours', ascending=False)

    print("Only", selected_user)
    return yes[:topk]
    
#print_watch_hours(topk=20, selected_user=USERS[0])